In [1]:
import taichi as ti
import src_ti.Simulation as Simulation
import importlib
importlib.reload(Simulation)
ti.init()

flow_kernel = ti.Vector.field(2, dtype=ti.i8, shape=5)
flow_kernel[0] = [0, 0]  # ORIGIN
flow_kernel[1] = [-1, 0] # UP
flow_kernel[2] = [0, 1]  # RIGHT
flow_kernel[3] = [1, 0]  # DOWN
flow_kernel[4] = [0, -1] # LEFT

num_com = 3


[Taichi] version 1.7.0, llvm 15.0.7, commit 6b563a2a, osx, python 3.10.8
[Taichi] Starting on arch=arm64


[I 10/22/23 11:02:36.459 11152024] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
f = ti.Vector.field(n=2, dtype=float, shape=(3, 3))
f.to_torch()


tensor([[[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]]])

In [3]:
ti.types.struct(**{'a': ti.f32, 'b': ti.types.vector(n=5, dtype=ti.f32)})

In [4]:

sim = Simulation.Simulation(
    'tst', (100,100),
    channels={
        'com': ti.types.vector(n=num_com, dtype=ti.f32),
        'muscles': ti.types.struct(
            flow=ti.types.vector(n=flow_kernel.shape[0], dtype=ti.f32),
            port=ti.f32,
            strength=ti.f32,),
        'macts': ti.types.struct(
            flow=ti.f32,
            port=ti.f32,
            mine=ti.f32),
        'gracts': ti.types.struct(
            flow=ti.types.vector(n=flow_kernel.shape[0], dtype=ti.f32),
            port=ti.f32,
            mine=ti.f32),
        'capital':  {'lims': (0,10)},
        'waste':    {'lims': (0,1)},
        'obstacle': {'lims': (0,1)},
        'port': {
            'lims': (-1,10),
            'metadata': {
                'num_resources': 2,
                'min_regen_amp': 0.5,
                'max_regen_amp': 2,
                'alpha_range': [0.4, 0.9],
                'beta_range': [0.8, 1.2],
                'num_sites_range': [2, 10]}},})

sim.init_data()

In [6]:
field = ti.Matrix.field(3, 4, ti.i32, shape=(256, 512))
field.shape  # (256, 512)
field.n      # 3
field.m      # 4

array = field.to_torch()
array.shape  # (256, 512, 3, 4)

# field.from_torch(array)  # the input array must be of shape (256, 512, 3, 4)

torch.Size([256, 512, 3, 4])

In [10]:
field = ti.Struct.field({'a': ti.i32, 'b': ti.types.vector(3, float)}, shape=(256, 512))
field.shape # (256, 512)

array_dict = field.to_torch()
array_dict

# array_dict['a'].shape # (256, 512)
# array_dict['b'].shape # (256, 512, 3)

# field.from_numpy(array_dict) # the input array must have the same keys as the field

{'a': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32),
 'b': tensor([[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
         

In [90]:
cfield = ti.Struct.field({
    "x": ti.f32,
    "a": ti.types.vector(2, dtype=ti.f32),
  }, shape = (2,2))

cfield.shape

(2, 2)